In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [33]:
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/2', creation_time=1716305867023, experiment_id='2', last_update_time=1716305867023, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/1', creation_time=1715878098390, experiment_id='1', last_update_time=1715878098390, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/0', creation_time=1715878098378, experiment_id='0', last_update_time=1715878098378, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
client.create_experiment(name="my-cool-experiment")

In [3]:
from mlflow.entities import ViewType

runs = client.search_runs(
       experiment_ids='1',
       filter_string="",
       run_view_type=ViewType.ACTIVE_ONLY,
       max_results=5,
       order_by=['metrics.rmse ASC']
   )
    

In [28]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f18cf67915c64d0a9ae802c3c034a0d0, rmse: 6.3184
run id: 659407035c974199b1fad48d6264d4f0, rmse: 6.3264
run id: ccd207c9af104162bf28e83bce41b31e, rmse: 6.3337
run id: 9261efee55974943a8d23b2f76b9478a, rmse: 6.3484
run id: 33554929f42b49709a88e64343fee096, rmse: 6.3620


In [29]:
runs

[<Run: data=<RunData: metrics={'rmse': 6.318445793399953}, params={'learning_rate': '0.09585355369315604',
  'max_depth': '30',
  'min_child_weight': '1.060597050922164',
  'objective': 'reg:linear',
  'reg_alpha': '0.018060244040060163',
  'reg_lambda': '0.011658731377413597',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "f18cf67915c64d0a9ae802c3c034a0d0", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2024-05-16 '
                              '17:46:40.243263", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.9.19", "data": "model.xgb", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "xgboost": {"xgb_version": '
                              '"2.0.3", "data": "model.xgb", "model_class": '
                 

In [58]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [35]:
run_id = "f18cf67915c64d0a9ae802c3c034a0d0"
model_uri = f'runs:/{run_id}/model'
mlflow.register_model(model_uri=model_uri, name="nyc_taxi_regressor")

Registered model 'nyc_taxi_regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc_taxi_regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716310075516, current_stage='None', description=None, last_updated_timestamp=1716310075516, name='nyc_taxi_regressor', run_id='f18cf67915c64d0a9ae802c3c034a0d0', run_link=None, source='/home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/1/f18cf67915c64d0a9ae802c3c034a0d0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [36]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716306678215, description=None, last_updated_timestamp=1716306678234, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716306678234, current_stage='None', description=None, last_updated_timestamp=1716306678234, name='nyc-taxi-regressor', run_id=None, run_link=None, source='run:/f18cf67915c64d0a9ae802c3c034a0d0/model', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-taxi-regressor', tags={}>,
 <RegisteredModel: aliases={}, creation_timestamp=1716303861797, description='', last_updated_timestamp=1716310075516, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716308703239, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-05-21', last_updated_timestamp=1716308782254, name='nyc_taxi_regressor', run_id=None, run_link=None, source='run:/f18cf67915c64d0a9ae802c3c034a0d0/model', status='READY', status_message=None, tags={}, user_id=None

In [52]:
model_name = 'nyc_taxi_regressor'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: Staging


/tmp/ipykernel_6312/701733819.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [38]:
client.transition_model_version_stage(
    name="nyc_taxi_regressor",
    version=4,
    stage="Staging",
)

/tmp/ipykernel_6312/2645119938.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716310075516, current_stage='Staging', description=None, last_updated_timestamp=1716310092836, name='nyc_taxi_regressor', run_id='f18cf67915c64d0a9ae802c3c034a0d0', run_link=None, source='/home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/1/f18cf67915c64d0a9ae802c3c034a0d0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [39]:
from datetime import datetime
model_version= 4
new_stage = "Staging"
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
   )

<ModelVersion: aliases=[], creation_timestamp=1716310075516, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2024-05-21', last_updated_timestamp=1716310097775, name='nyc_taxi_regressor', run_id='f18cf67915c64d0a9ae802c3c034a0d0', run_link=None, source='/home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/1/f18cf67915c64d0a9ae802c3c034a0d0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [40]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [20]:
!ls ../../data

green_tripdata_2021-01.parquet	yellow_tripdata_2023-01.parquet
green_tripdata_2021-02.parquet	yellow_tripdata_2023-02.parquet
green_tripdata_2021-03.parquet


In [21]:
df = read_dataframe("../../data/green_tripdata_2021-03.parquet")

In [56]:
len(df.columns)

22

In [22]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/ubuntu/mlopszoomcamp/02-experiment-tracking/preprocessor'

In [47]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [48]:
X_test = preprocess(df, dv)

In [49]:
target = "duration"
y_test = df[target].values


In [50]:
#!mlflow experiments list

In [57]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

OSError: No such file or directory: '/home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/1/f18cf67915c64d0a9ae802c3c034a0d0/artifacts/model/.'

In [46]:
!ls /home/ubuntu/mlopszoomcamp/02-experiment-tracking/mlruns/1/f18cf67915c64d0a9ae802c3c034a0d0/artifacts/models_mlflow

MLmodel  conda.yaml  metadata  model.xgb  python_env.yaml  requirements.txt


In [34]:
!pwd

/home/ubuntu/mlopszoomcamp/02-experiment-tracking


In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

In [55]:
model_name

'nyc_taxi_regressor'

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)